In [5]:
from datetime import datetime
import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
from PIL import Image
import html
from markdownify import markdownify 
import json

endpoint_url = "https://query.wikidata.org/sparql"

query_img ="""
SELECT DISTINCT ?artwork ?artworkLabel ?image ?width ?height ?artworkDescription (YEAR(?incep) AS ?year) (STRAFTER(STR(?artwork), "/entity/") AS ?qid) WHERE {
  ?artwork wdt:P31 wd:Q3305213;
    wdt:P170 wd:Q41406;
    wdt:P18 ?image;
    wdt:P2048 ?height;
    wdt:P2049 ?width;
    wdt:P571 ?incep.
  FILTER(?width > ?height)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,[AUTO_LANGUAGE]". }
}
LIMIT 25
"""


def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_delay(date):
    try:
        date = datetime.datetime.strptime(date, '%a, %d %b %Y %H:%M:%S GMT')
        timeout = int((date - datetime.datetime.now()).total_seconds())
    except ValueError:
        timeout = int(date)
    return timeout


def get_img(partOfItem_id):
    print("")
    print("<link rel='stylesheet' href='./section-fullimg.css'>")

    q = ""
    print("<div class='navig'>")

    print("<input type='button' class='btn btn-secondary arrow arrowleft' onclick='openPrev()' value='&#10092;'/>")

    #print("<div class='stop-scrolling'>")

    if partOfItem_id:
        q = query_img.replace("<placeholder>", "?partOfStatement cpsps:P6 cps:"+partOfItem_id+".")
    else:
        q = query_img.replace("<placeholder>","")
    results_img = run_query(endpoint_url, q)


    results = []
    for item in results_img["results"]["bindings"]:
        results.append({
            "year": item.get("year", {}).get("value", "Unknown"),
            "name": item.get("artworkLabel", {}).get("value", "No title"),
            "link": item.get("artwork", {}).get("value", "#"),
            "alt": item.get("artworkDescription", {}).get("value", "Painting"),
            "image": item["image"]["value"],
            "qid": item.get("qid", {}).get("value", ""),
            "width": item.get("width", {}).get("value", "?"),
            "height": item.get("height", {}).get("value", "?")

        })

    json_data = json.dumps(results)

    print(f'<img id="img_full" src="" alt="">')

    print("<input type='button' class='btn btn-secondary arrow arrowright' onclick='openNext()' value='&#10093;'/>")

    print(f'''<script>
        const paintings = {json_data};
        let currentIndex = 0;

        function updateTable() {{
          const painting = paintings[currentIndex];
          document.getElementById("paintingYear").innerText = painting.year;
          document.getElementById("paintingName").innerText = painting.name;
          document.getElementById("paintingLink").href = painting.link;
          document.getElementById("img_full").src = painting.image;
          document.getElementById("img_full").alt = painting.alt;
          //document.getElementById("paintingID").innerText = painting.qid;
          document.getElementById("paintingSize").innerText = painting.width + ' cm x ' + painting.height + ' cm';



        }}

        function openNext() {{
          currentIndex = (currentIndex + 1) % paintings.length;
          updateTable();
        }}

        function openPrev() {{
          currentIndex = (currentIndex - 1 + paintings.length) % paintings.length;
          updateTable();
        }}

        document.addEventListener("DOMContentLoaded", updateTable);
    </script>''')

    print("</div>")



    print('<div id="infogrid"><div id="paintingName"></div>')
    print('<div id="paintingYear"></div>')
    print('<span id="paintingSize"></span>')
    print('<span><a id="paintingLink">Wikidata</a></span></div>')



get_img("")


<link rel='stylesheet' href='./section-fullimg.css'>
<div class='navig'>
<input type='button' class='btn btn-secondary arrow arrowleft' onclick='openPrev()' value='&#10092;'/>
<img id="img_full" src="" alt="">
<input type='button' class='btn btn-secondary arrow arrowright' onclick='openNext()' value='&#10093;'/>
<script>
        const paintings = [{"year": "1882", "name": "Evening Atmosphere at Sea", "link": "http://www.wikidata.org/entity/Q18889825", "alt": "painting by Edvard Munch", "image": "http://commons.wikimedia.org/wiki/Special:FilePath/Edvard%20Munch%20-%20Evening%20Atmosphere%20at%20Sea.jpg", "qid": "Q18889825", "width": "23.5", "height": "21"}, {"year": "1902", "name": "Four Girls in \u00c5sg\u00e5rdstrand", "link": "http://www.wikidata.org/entity/Q18889886", "alt": "painting by Edvard Munch (1902 - 1)", "image": "http://commons.wikimedia.org/wiki/Special:FilePath/Stuttgart%20-%20Neue%20Staatsgalerie%20%2835518740560%29.jpg", "qid": "Q18889886", "width": "125.5", "height":

In [6]:
#get_text("Q209")

In [7]:
#get_img("")

<hr>

In [8]:
#get_graph()